In [1]:
import csv
import networkx as nx
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
from sklearn.linear_model import LogisticRegression

In [1]:
!nvidia-smi

Fri May 27 16:58:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           On   | 00000000:1B:00.0 Off |                  N/A |
| 41%   48C    P2    65W / 280W |    920MiB / 24220MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           On   | 00000000:1E:00.0 Off |                  N/A |
| 41%   

In [2]:
# prepare dataset

f = open('train.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
trains = []
for line in reader:
    trains.append(line)
f.close()    
print("trains", len(trains))

f = open('node_ingredient.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
node_ingredients = []
for line in reader:
    node_ingredients.append(line)
f.close()    
print("node_ingredients", len(node_ingredients))
num_ing = len(node_ingredients)

# f = open('validation_classification_question.csv', 'r', encoding='utf-8')
# reader = csv.reader(f)
# val_cls_q = []
# for line in reader:
#     val_cls_q.append(line)
# f.close()
# print("val_cls_q", len(val_cls_q))

# f = open('validation_classification_answer.csv', 'r', encoding='utf-8')
# reader = csv.reader(f)
# val_cls_a = []
# for line in reader:
#     val_cls_a.append(line)
# f.close()
# print("val_cls_a", len(val_cls_a))

f = open('validation_completion_question.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
val_cpt_q = []
for line in reader:
    val_cpt_q.append(line)
f.close()
print("val_cpt_q", len(val_cpt_q))

f = open('validation_completion_answer.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
val_cpt_a = []
for line in reader:
    val_cpt_a.append(line)
f.close()
print("val_cpt_a", len(val_cpt_a))

trains 23547
node_ingredients 6714
val_cpt_q 7848
val_cpt_a 7848


## Completion Task : Method 0
find neighbors of largest weight (no use of embedding)

In [39]:
# make graph of ingredients
G = nx.Graph()
for i in range(len(node_ingredients)):
    G.add_node(str(i))
    
print(G.number_of_nodes())

for data in trains:
    for i in range(len(data) - 2):
        for j in range(i+1, len(data) - 1):
            if G.has_edge(data[i], data[j]):
                G[data[i]][data[j]]['weight'] += 1
            else:
                G.add_edge(data[i], data[j], weight=1)

print(G.number_of_edges())

6714
355816


In [40]:
acc = 0
for i, data in enumerate(val_cpt_q):
#     print(data)
    weight_dict = {}
    for node in data:
        for adv, w in G.adj[node].items():
            if adv in weight_dict.keys():
                weight_dict[adv] += w['weight']
            else:
                weight_dict[adv] = w['weight']
    for node in data:
        if node in weight_dict.keys():
            del weight_dict[node]
    
    weight_dict = sorted(weight_dict.items(), key=(lambda x: x[1]), reverse=True)
    
#     print(weight_dict)

    print(weight_dict[0][0], val_cpt_a[i][0])
    if weight_dict[0][0] == val_cpt_a[i][0]:
        acc += 1
        
print("accuracy: ", acc / len(val_cpt_q) * 100, "%")

937 2061
4799 6421
2945 2945
5377 281
2945 5119
937 5577
937 4059
937 469
2945 1004
5377 2032
937 4945
937 3826
937 3259
937 937
937 3915
937 1046
937 2064
937 2945
5536 1435
937 2122
937 3978
5536 4543
5536 1667
937 2174
5536 2945
5377 3238
5536 5377
937 1816
5536 4799
937 3146
5377 5918
5536 2651
5536 5136
937 5110
5536 3146
5377 4260
5536 2567
5536 1354
5536 2518
5536 791
937 3556
937 2759
5377 4582
937 2813
937 2945
6187 1476
937 962
937 3653
3978 2813
937 579
5377 3885
937 1390
5536 2710
937 4266
937 5459
937 1953
5536 5377
5536 4195
937 4335
5536 1858
937 6095
5536 59
5377 4162
937 5176
5536 2792
937 6438
5536 2122
937 5377
937 530
937 6327
5536 59
5377 5377
937 937
5536 5204
5536 4038
937 3725
937 4303
5536 2518
937 5377
937 937
5536 401
937 3645
937 4582
937 2259
937 937
937 3564
5377 2757
937 5136
5536 2064
937 4561
1308 895
937 3324
937 3953
937 530
937 1597
5536 2351
937 3622
2945 1920
937 5654
937 6484
937 1322
937 6476
2945 1790
937 2758
937 5459
1308 5518
937 1476
937 338

5536 4582
5377 6349
937 2759
937 3223
937 3060
5536 2483
5536 5774
5377 4266
937 2009
5536 3596
937 937
937 2122
937 1112
5377 4685
937 937
937 992
2945 4692
937 909
937 4720
5536 5536
937 4853
937 6187
937 946
937 6471
937 4799
4799 2408
4799 1914
937 937
5536 5162
937 1816
1308 1364
5377 2338
2945 2833
4799 3001
937 5377
937 1396
937 681
937 4013
937 1790
5377 5162
937 5512
937 5815
937 5532
937 2138
937 6126
937 937
937 3934
937 4349
6187 3148
5536 3886
5536 4796
6187 4009
5536 2122
937 170
937 4207
937 4099
5536 4945
937 2122
937 1308
937 1354
937 2319
937 5882
6187 5884
937 937
937 6187
937 1842
5536 6187
5536 5249
937 3447
5377 59
937 3228
937 3223
937 2945
6187 5884
5377 4038
937 6276
5536 4639
937 3507
5536 5698
5536 5648
937 6426
1308 2710
2945 909
5377 4799
937 6361
5377 5377
937 4506
5536 2945
937 4945
2945 2809
5536 6187
937 4982
937 4622
937 2809
937 6187
5536 4506
937 6619
937 4266
937 6496
5377 2866
937 2122
937 6187
2945 1048
937 701
937 4361
3978 5216
937 1799
937 2207

2945 399
937 937
5536 1554
937 753
937 1903
6187 5233
5536 6666
937 1935
937 1679
2945 4178
937 3733
5536 2518
5377 3070
5536 5536
5377 868
937 3064
937 3978
937 2564
937 5432
937 4672
937 59
937 1111
937 6187
5536 4462
937 3607
5377 2885
937 2414
937 3934
5377 5405
5377 2203
5536 5884
937 5675
5536 6187
937 1308
937 1676
5536 4917
937 5377
937 5536
937 1676
937 4642
937 937
5536 527
937 3978
937 5648
937 2809
937 1953
937 937
937 937
5377 3885
937 6159
937 1816
6187 5908
937 4909
5536 4856
5377 778
937 4672
5377 1735
6187 2813
937 1476
937 909
937 579
5536 3060
937 5774
937 4799
937 5536
937 937
937 4917
937 5377
5536 2442
5536 5648
937 317
5536 2332
5536 2332
5536 2956
937 4979
5536 5293
6187 579
937 4812
5377 802
937 5249
5536 479
5536 2813
5536 2656
5377 6187
937 1953
5536 2945
2945 2375
937 6473
5536 80
937 964
937 6036
937 1679
937 2813
5536 1496
2945 4677
937 4791
2945 5884
937 1468
5536 1476
937 6456
937 6433
937 4355
937 832
937 6496
937 4999
937 5882
2945 4649
937 2945
5536 3

5377 5377
937 3824
5377 5888
937 2701
937 937
5377 1838
937 3595
937 1816
5536 1743
5536 399
4799 6648
2945 4162
937 3597
5536 2455
937 3597
5536 4799
5536 4243
937 6429
937 4945
937 2919
937 802
937 4478
937 2122
937 5884
5536 5803
5536 6476
937 4539
937 1909
5377 1922
937 47
937 1785
937 5570
5536 3401
937 3622
5536 2351
937 1308
5377 2518
5536 167
937 937
937 2485
937 4289
937 4799
937 6131
937 4060
937 5650
5536 3886
937 3483
937 2956
5536 6711
5536 5536
937 203
937 3371
937 2528
937 2945
937 937
937 4843
937 1454
5536 5249
937 5455
5377 4027
4799 2809
937 802
5377 6610
4799 167
937 3429
5536 2619
937 5536
5377 4843
5536 1390
937 1816
937 1112
4799 694
937 1189
937 167
5536 2945
937 4582
937 3833
937 1785
5536 5632
937 2647
937 937
937 4543
937 1150
937 1679
5536 1390
937 2078
937 3146
937 3777
3978 281
937 74
5536 579
937 5536
937 3070
937 5648
937 1198
5377 5882
937 1955
937 2564
937 3202
937 5251
937 3978
5377 6496
937 2296
6187 2122
937 5609
5536 1308
5536 1573
5536 3742
937 33

2945 4048
937 684
6187 5377
937 6502
2945 3822
937 2945
5377 2122
937 5136
937 3653
937 4475
5536 1914
937 1308
937 3833
937 2651
1308 909
937 962
937 6253
937 4266
937 6516
5536 1679
5377 1842
6187 958
937 4618
5536 6241
5377 962
937 2937
937 3653
5377 265
5536 5251
937 4561
937 4917
937 2945
1308 1308
937 4327
937 5117
937 4672
6187 1308
937 720
937 773
937 773
1308 2235
5536 5536
937 937
1308 1626
937 6384
937 3238
5536 4355
6187 5459
5536 1785
5536 2518
937 4632
5536 1318
937 3396
937 1909
937 937
937 3228
5536 2813
937 2458
5377 6187
937 3789
937 308
5377 3148
937 323
937 4038
937 4060
937 937
937 2442
937 4632
5536 2813
937 5536
937 992
5377 992
4799 167
937 2809
5536 4059
937 4595
937 3144
937 937
5536 2809
937 2956
937 5249
937 210
6187 265
937 3205
5536 581
5536 1202
5536 4009
5536 3749
5536 552
937 3097
937 3036
2945 4162
937 2192
937 2925
937 2809
937 2616
937 167
5536 3240
5536 3653
937 3622
937 2945
6187 6610
937 3242
5536 2464
937 1476
937 3646
937 937
937 5405
937 4884
2

937 59
937 2375
937 897
937 1939
937 3978
937 5456
937 3146
937 3777


KeyboardInterrupt: 

## Completion Task : Method 1
cosine similarity of embedding

In [3]:
def cos_sim(X,y):
    return np.dot(X, y) # / (np.linalg.norm(X, axis=1) * np.linalg.norm(y))

In [4]:
def make_embedding(filename, hiddensize):
    f = open(filename, 'r', encoding='utf-8')
    reader = csv.reader(f)
    f.readline()
    embedding = np.zeros((6714, hiddensize))
    for line in reader:
        i = int(line[1])
        j = 0
        for node in line[2][2:-1].split(' '):
            if node != '':
                embedding[i][j] = float(node.strip())
                j += 1

    f.close()
    return embedding

In [5]:
embp1q10 = make_embedding('Embedding/Embp1q10.csv', 64)
embp1q100 = make_embedding('Embedding/Embp1q2.csv', 64)
svd128 = make_embedding('Embedding/SVD128.csv', 128)
svd64 = make_embedding('Embedding/SVD64.csv', 64)
svd32 = make_embedding('Embedding/SVD32.csv', 32)

In [6]:
# method 1-1: similarity of average

def similarityBased(embedding):
    acc = 0
    for i, data in tqdm(enumerate(val_cpt_q)):
        nodes = [int(node) for node in data]
        avg_node = np.average(embedding[nodes][:], axis=0)
        sims = cos_sim(embedding, avg_node)
        ranks = np.argsort(-sims)

        target = int(val_cpt_a[i][0])

    #     print("input: ", data)
    #     print("target: ", target)
    #     print("estimation: ", ranks[0:10])
    #     print("rank: ", np.where(ranks == target))
    #     print("")

        j = 0
        while ranks[j] in data:
            j += 1

        estimation = ranks[j]

        if estimation == target:
            acc += 1
            
    return acc / len(val_cpt_q)

In [7]:
print(similarityBased(embp1q10))
print(similarityBased(embp1q100))
print(similarityBased(svd128))
print(similarityBased(svd64))
print(similarityBased(svd32))

7848it [00:08, 912.12it/s] 
108it [00:00, 1077.17it/s]

0.0061162079510703364


7848it [00:07, 981.35it/s] 
112it [00:00, 1115.26it/s]

0.004714576962283384


7848it [00:08, 930.82it/s] 
112it [00:00, 1113.74it/s]

0.04001019367991845


7848it [00:08, 977.69it/s] 
80it [00:00, 794.77it/s]

0.0414118246687054


7848it [00:08, 965.39it/s] 

0.02408256880733945


In [185]:
# random
print(1 / 6714 * 100, '%')

0.014894250819183795 %


In [18]:
# method 1-2: average of similarity

def similarityBased2(embedding):
    acc = 0
    for i, data in tqdm(enumerate(val_cpt_q)):
        nodes = [int(node) for node in data]
        sims = []
        for node in nodes:
            sims.append(cos_sim(embedding, embedding[node]))
        sims = np.stack(sims, axis=0)
        avg_sim = np.average(sims, axis=0)
        ranks = np.argsort(-avg_sim)

        target = int(val_cpt_a[i][0])

        j = 0
        while ranks[j] in data:
            j += 1

        estimation = ranks[j]

        if estimation == target:
            acc += 1

    return acc / len(val_cpt_q)

In [19]:
print(similarityBased2(embp1q10))
print(similarityBased2(svd128))
print(similarityBased2(svd64))
print(similarityBased2(svd32))

0it [00:00, ?it/s]/home/lym7505/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
7848it [01:07, 116.99it/s]
6it [00:00, 56.00it/s]

0.00191131498470948


7848it [01:55, 68.07it/s]
11it [00:00, 105.48it/s]

0.006625891946992864


7848it [01:04, 121.86it/s]
14it [00:00, 132.34it/s]

0.011213047910295617


7848it [00:49, 157.66it/s]

0.014780835881753314


## Logistic Regression

In [108]:
def make_dataset(trainset):
    xs = []
    ys = []
    for data in tqdm(trainset):
        nodes = [int(node) for node in data[:-1]]
        x = np.zeros((len(nodes), num_ing))
        y = np.zeros(len(nodes))
        [1,2,3,4,5]
        1,2,3,4  5
        1,2,3,5  4
        
        for i, node in enumerate(nodes):
            other_nodes = [nd for nd in nodes if nd != node]
            for other in other_nodes:
                x[i][other] = 1
                y[i] = node
        xs.append(x)
        ys.append(y)
    train_x = np.concatenate(xs, axis=0)
    train_y = np.concatenate(ys, axis=0)
    return train_x, train_y

In [114]:
train_x, train_y = make_dataset(trains)

100%|██████████| 23547/23547 [00:06<00:00, 3542.06it/s]


In [113]:
clf = LogisticRegression(penalty='l2', max_iter=10, verbose=True).fit(train_x, train_y)
clf.score(train_x, train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 45.0min finished


0.10297918006462584

In [124]:
def make_valset(question, answer):
    val_x = np.zeros((len(question), num_ing))
    val_y = np.zeros(len(question))
    for i in range(len(question)):
        nodes = [int(node) for node in question[i]]
        target = int(answer[i][0])
        for node in nodes:
            val_x[i][node] = 1
        val_y[i] = target
        
    return val_x, val_y

In [125]:
val_x, val_y = make_valset(val_cpt_q, val_cpt_a)

In [127]:
clf.score(val_x, val_y)

0.09174311926605505

## RNN

In [191]:
class CompleteRNN(nn.Module):
    def __init__(self, input_size, n_hidden, n_class):
        super(CompleteRNN, self).__init__()

        self.rnn = nn.RNN(input_size=input_size, hidden_size=n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(torch.float32))
        self.b = nn.Parameter(torch.randn([n_class]).type(torch.float32))
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        X = X.type(torch.float32)
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1]
        model = torch.mm(outputs, self.W) + self.b
#         model = torch.softmax(model, dim=1)
        return model

In [192]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, train_list, embedding):
        self.input = []
        self.target = [] # missing ingredient
        
        for data in train_list:
            nodes = [int(node) for node in data[0:-1]]
            for node in nodes:
                other_nodes = [nd for nd in nodes if nd != node]
                if len(other_nodes) == 0:
                    continue
                examples = []
                for other in other_nodes:
                    examples.append(embedding[other])
                self.input.append(np.stack(examples))
                self.target.append(node)                

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        inp = self.input[idx]
        target = self.target[idx]
        return inp, target

In [193]:
# train_loader = torch.utils.data.DataLoader(
#     TrainDataset(trains, transform=torchvision.transforms.ToTensor()), 
#     batch_size=32, 
#     shuffle=True
# )

batch_size = 1

train_loader = torch.utils.data.DataLoader(
    TrainDataset(trains, svd32), 
    batch_size=batch_size, 
    shuffle=True
)

In [194]:
class ValDataset(torch.utils.data.Dataset):
    def __init__(self, val_question, val_answer, embedding):
        self.input = []
        self.target = [] # missing ingredient
        
        for i in range(len(val_question)):
            nodes = [int(node) for node in val_question[i]]
            examples = []
            for node in nodes:
                examples.append(embedding[node])
            self.input.append(np.stack(examples))
            self.target.append(int(val_answer[i][0]))

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        inp = self.input[idx]
        target = self.target[idx]
        return inp, target

In [195]:
val_loader = torch.utils.data.DataLoader(
    ValDataset(val_cpt_q, val_cpt_a, svd32), 
    batch_size=batch_size, 
    shuffle=False
)

In [196]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [198]:
input_size = 32
n_hidden = 64
model = CompleteRNN(input_size=input_size, n_hidden=n_hidden, n_class=6714)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,  milestones=[10000, 20000], gamma=0.1)

for epoch in range(10):
    losses = AverageMeter()
    acces = AverageMeter()
    num_datas = 0
    for i, (input_data, label) in enumerate(train_loader):
#         print(i, data, label)
        hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
        output = model(hidden, input_data)
        loss = criterion(output, label)
        pred = torch.argmax(output, dim=1)
        
        losses.update(loss.item(), input_data.size(0))
        pred = pred.numpy()
        label = label.numpy()
        acces.update((pred == label).sum(), input_data.size(0))
        if i % 50000 == 0:
            print(epoch, i, losses.avg, acces.avg)
#             losses.reset()
#             acces.reset()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    print("train", epoch, losses.avg, acces.avg)
        
    acc = 0
    with torch.no_grad():
        for i, (input_data, label) in enumerate(val_loader):
            hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
            output = model(hidden, input_data)
            pred = torch.argmax(output, dim=1)
            if pred.item() == label.item():
                acc += 1
    print("validation", epoch, acc / len(val_loader))
        

0 0 16.93003273010254 0.0
0 50000 7.90941015814703 0.028039439211215777
0 100000 7.4642755396515055 0.03168968310316897
0 150000 7.286343715739898 0.03348644342371051
0 200000 7.178960488742359 0.03468482657586712
0 250000 7.106298201371221 0.03559585761656953
train 0 7.102274708677027 0.035657354797979796
validation 0 0.03567787971457696
1 0 5.262133598327637 0.0
1 50000 6.7709534023430855 0.038939221215575685
1 100000 6.739326040120917 0.04052959470405296
1 150000 6.72929992668136 0.0406463956906954
1 200000 6.721165427088099 0.04062479687601562
1 250000 6.713961237400079 0.040523837904648384
train 1 6.713073569043737 0.04064867424242424
validation 1 0.04217635066258919
2 0 3.966636896133423 0.0
2 50000 6.639087900927357 0.04079918401631968
2 100000 6.627683479136641 0.04146958530414696
2 150000 6.627158396502382 0.04174638835741095
2 200000 6.6277488939002795 0.04201478992605037
2 250000 6.6263786295147975 0.0420998316006736
train 2 6.625761048480247 0.04211647727272727
validation 2

In [162]:
acc = 0
with torch.no_grad():
    for i, (input_data, label) in enumerate(val_loader):
        hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
        output = model(hidden, input_data)
        pred = torch.argmax(output, dim=1)
        if pred.item() == label.item():
            acc += 1
print(acc / len(val_loader))

0.03835372069317024


## Frequent Item 

Interest(I -> j) = Confidence(I -> j) - Pr[j]

1. Get Pr[j] for all j


In [6]:
from collections import defaultdict

singles = defaultdict(int)
pairs = defaultdict(int)
triples = defaultdict(int)
quadraples = defaultdict(int)
quintuples = defaultdict(int)

for data in tqdm(trains):
    nodes = [int(node) for node in data[0:-1]]
    nodes = sorted(nodes)
    for i in range(len(nodes)):
        singles[nodes[i]] += 1
            
    for i in range(len(nodes) - 1):
        for j in range(i+1, len(nodes)):
            pairs[(nodes[i], nodes[j])] += 1
            
    for i in range(len(nodes) - 2):
        for j in range(i+1, len(nodes) - 1):
            for k in range(j+1, len(nodes)):
                triples[(nodes[i], nodes[j], nodes[k])] += 1
                
    for i in range(len(nodes) - 3):
        for j in range(i+1, len(nodes) - 2):
            for k in range(j+1, len(nodes) - 1):
                for l in range(k+1, len(nodes)):
                    quadraples[(nodes[i], nodes[j], nodes[k], nodes[l])] += 1
                    
    for i in range(len(nodes) - 4):
        for j in range(i+1, len(nodes) - 3):
            for k in range(j+1, len(nodes) - 2):
                for l in range(k+1, len(nodes) - 1):
                    for t in range(l+1, len(nodes)):
                        quintuples[(nodes[i], nodes[j], nodes[k], nodes[l], nodes[t])] += 1

print(len(singles))
print(len(pairs))
print(len(triples))
print(len(quadraples))

100%|██████████| 23547/23547 [01:42<00:00, 228.90it/s]

5859
355816
3773815
18745846


In [9]:
def interest(I, j):
#     if pairs[tuple(sorted(I))] == 0:
#         return 0
    
    confidence = triples[tuple(sorted(I + [j]))] / (pairs[tuple(sorted(I))] + 0.01)
#     print(confidence, singles[j] / len(trains))
    return confidence
#     return abs(confidence - singles[j] / len(trains))
#     return abs(confidence + 0.1 * math.log10((singles[j] + 1) / len(trains)))
#     return confidence / (singles[j] + 0.01)

In [20]:
def interest(I, j):
    if len(I) == 4:
        confidence = quintuples[tuple(sorted(I + [j]))] / (quadraples[tuple(sorted(I))] + 0.01)
    elif len(I) == 3:
        confidence = quadraples[tuple(sorted(I + [j]))] / (triples[tuple(sorted(I))] + 0.01)
    elif len(I) == 2:
        confidence = triples[tuple(sorted(I + [j]))] / (pairs[tuple(sorted(I))] + 0.01)
    else:
        print("error")
    return confidence

In [22]:
for id, data in enumerate(val_cpt_q[:10]):
    target = int(val_cpt_a[id][0])
    nodes = [int(node) for node in data]
    print(nodes, target)
    value1s = 0
    value2s = 0
    for i in range(len(nodes) - 3):
        for j in range(i+1, len(nodes) - 2):
            for k in range(j+1, len(nodes) - 1):
                for l in range(k+1, len(nodes)):
                    value1 = interest([nodes[i], nodes[j], nodes[k], nodes[l]], target)
                    value2 = interest([nodes[i], nodes[j], nodes[k], nodes[l]], target + 10)
                    value1s += value1
                    value2s += value2
#                     print(value1, value2)
    print(value1s, value2s)
    

[59, 233, 1308, 2338, 3332, 6683] 2061
0.9900990099009901 0.0
[937, 1138, 1240, 1914, 2809, 3071, 3228, 3614, 4191, 5140, 5554, 5648, 5774, 5841, 5866, 6294] 6421
6.798599216639484 0.0
[937, 1308, 1476, 2710, 4342, 4799, 4917] 2945
6.009293233087514 0.10808663281854261
[154, 628, 937, 1137, 1396, 1476, 1856, 2483, 2813, 3886, 4038, 4349, 5536, 5648, 6187, 6434] 281
165.74527722933541 0.0
[80, 937, 2518, 3184, 3396, 4664, 4884, 5377, 5536, 5654, 6187] 5119
37.21250862358252 0.0
[182, 738, 1321, 1454, 1922, 2132, 2518, 3144, 4757, 4945, 5110, 5908, 6264, 6276, 6352, 6371] 5577
0.0 0.0
[203, 323, 2122, 2564, 3556, 3832, 5536, 5661, 5731, 5882, 6294, 6422, 6669] 4059
36.314986214841696 0.0
[628, 1554, 1785, 2406, 2945, 4945, 5051, 5536, 5908, 6187, 6422] 469
5.327005768999637 0.0
[248, 399, 526, 552, 937, 1308, 2061, 2106, 2122, 2221, 2764, 3332, 4707, 4824, 5621, 6469] 1004
0.0 0.0
[265, 937, 962, 1169, 2760, 2813, 3146, 3857, 4619] 2032
0.0 0.0


In [23]:
for id, data in enumerate(val_cpt_q[:10]):
    target = int(val_cpt_a[id][0])
    nodes = [int(node) for node in data]
    print(nodes, target)
    value1s = 0
    value2s = 0
    for i in range(len(nodes) - 2):
        for j in range(i+1, len(nodes) - 1):
            for k in range(j+1, len(nodes)):
                value1 = interest([nodes[i], nodes[j], nodes[k]], target)
                value2 = interest([nodes[i], nodes[j], nodes[k]], target + 10)
                value1s += value1
                value2s += value2
    print(value1s, value2s)
    

[59, 233, 1308, 2338, 3332, 6683] 2061
5.589624070040327 0.0
[937, 1138, 1240, 1914, 2809, 3071, 3228, 3614, 4191, 5140, 5554, 5648, 5774, 5841, 5866, 6294] 6421
3.5859448757283316 0.0
[937, 1308, 1476, 2710, 4342, 4799, 4917] 2945
5.498924920568486 0.17469472156967886
[154, 628, 937, 1137, 1396, 1476, 1856, 2483, 2813, 3886, 4038, 4349, 5536, 5648, 6187, 6434] 281
71.60908709526254 0.00861994655633135
[80, 937, 2518, 3184, 3396, 4664, 4884, 5377, 5536, 5654, 6187] 5119
25.622059288046273 0.0
[182, 738, 1321, 1454, 1922, 2132, 2518, 3144, 4757, 4945, 5110, 5908, 6264, 6276, 6352, 6371] 5577
0.0 0.0
[203, 323, 2122, 2564, 3556, 3832, 5536, 5661, 5731, 5882, 6294, 6422, 6669] 4059
21.96356333718404 0.1996007984031936
[628, 1554, 1785, 2406, 2945, 4945, 5051, 5536, 5908, 6187, 6422] 469
4.185906106545778 0.035287096651116526
[248, 399, 526, 552, 937, 1308, 2061, 2106, 2122, 2221, 2764, 3332, 4707, 4824, 5621, 6469] 1004
0.49861526470279327 0.0
[265, 937, 962, 1169, 2760, 2813, 3146, 3857,

In [10]:
acc = 0

for id, data in enumerate(val_cpt_q):
    nodes = [int(node) for node in data]
#     print(nodes)
    interest_list = [0] * num_ing
    for k in range(num_ing):
        temp = 0
        for i in range(len(nodes) - 1):
            for j in range(i+1, len(nodes)):
                interest_list[k] += interest([nodes[i], nodes[j]], k)
                temp += 1
        interest_list[k] = interest_list[k] / temp
#     for i in range(len(interest_list)):
#         if interest_list[i] != 0:
#             print(i, interest_list[i])
    estimation = max(range(len(interest_list)), key=lambda i: interest_list[i])
    target = int(val_cpt_a[id][0])
    print(estimation, interest_list[estimation], singles[estimation], "/", target, interest_list[target], singles[target])
    if estimation == target:
        acc += 1
        
    if id % 50 == 0 and id > 0:
        print("accuracy: ", acc / (id + 1) * 100, "%")
    
print(len(val_cpt_q))
print("accuracy: ", acc / len(val_cpt_q) * 100, "%")

2061 0.3764055455067568 305 / 2061 0.3764055455067568 305
937 0.33036686927066494 10683 / 6421 0.00882927832026951 125
937 0.4949999734028248 10683 / 2945 0.15922771589134266 4347
937 0.5813254404586624 10683 / 281 0.10392155858874519 778
937 0.4582874193304521 10683 / 5119 0.15920627213226213 933
937 0.4109100714884535 10683 / 5577 0.0 35
937 0.3582420077393694 10683 / 4059 0.06533764229675573 320
937 0.4141014140075587 10683 / 469 0.03002235208382128 518
3978 0.20999923186976968 1928 / 1004 0.0045783677949261855 88
937 0.40530304386259974 10683 / 2032 0.001086988493154972 28
937 0.49793164946527363 10683 / 4945 0.0738836795400104 1148
937 0.3826039176176616 10683 / 3826 0.11066434087609092 703


KeyboardInterrupt: 

In [ ]:
interest(I -> j) = confidence(I -> j) - Pr[j]